# Task 2

### Load Drug dataset

In [24]:
import pandas as pd

drug_dataset = pd.read_csv('./data/drug200.csv')

drug_dataset.head()


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [30]:
import matplotlib.pyplot as plt

sp = drug_dataset['Drug'].value_counts().plot(title='Drug instance count', figsize=(18, 6),
                                         kind='barh')

plt.savefig('./out/drug-distribution.pdf')